In [7]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show, output_file, save, push_notebook, curdoc
from bokeh.plotting import figure, curdoc, gridplot
from bokeh.models import (DateRangeSlider, ColumnDataSource, HoverTool, CustomJS, 
                          Slider, Range1d, FactorRange, Legend, Label, 
                          LabelSet, ColorBar, NumeralTickFormatter, 
                          DatetimeTickFormatter, Toggle, CheckboxGroup, 
                          RadioButtonGroup, TextInput, Button, Div, Tabs)
from bokeh.layouts import row, column, gridplot, layout
from bokeh.palettes import HighContrast3, Viridis256, Category20, Inferno256, Cividis256, Turbo256
from bokeh.transform import factor_cmap, dodge, linear_cmap, log_cmap
from bokeh.events import DoubleTap, MouseMove, PanStart, Tap
# from bokeh.tile_providers import get_provider, Vendors
from bokeh.embed import components, file_html
from bokeh.resources import CDN
from bokeh.themes import Theme
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource, HoverTool, RangeTool, Range1d, BoxAnnotation
from bokeh.io import output_file, curdoc, show
from bokeh.models import ColumnDataSource, HoverTool, Button, CustomJS, Div, RangeTool, BoxAnnotation
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot
from bokeh.palettes import Category10, Category20
import numpy as np
import pandas as pd

In [8]:
age_opts = ['6m', '9m', '12m', '18m']
loc_opts = ['STL', 'UNC', 'PHI']
clusters = [1, 2, 3]

x = np.linspace(1, 1000, 1000)
y = np.random.normal(5, 1, len(x))
c = np.random.choice(clusters, size = len(x))
age = np.random.choice(age_opts, size = len(x))
loc = np.random.choice(loc_opts, size = len(x))

data = pd.DataFrame({
    'x': x,
    'y': y,
    'c': c,
    'age' : age, 
    'loc' : loc
}, index=pd.Index(range(1, len(x) + 1), name='frame'))

In [9]:
output_notebook()
def prepare_wedge_data(column_name, colors):
    counts = data[column_name].value_counts()
    categories = sorted(counts.index)
    sizes = counts.loc[categories].values

    angles = np.linspace(0, 2 * np.pi, len(categories) + 1)
    start_angles = angles[:-1]
    end_angles = angles[1:]

    return pd.DataFrame({
        'start_angle': start_angles,
        'end_angle': end_angles,
        'color': colors,
        column_name: categories,
        'value': sizes
    })

source = ColumnDataSource(data)  # Reset index to include it as a column

# Define colors based on clusters
colors = Category10[3]  # Use a color palette for up to 10 clusters
data['color'] = data['c'].map(lambda cluster: colors[cluster-1])
source.add(data['color'], 'color')

# Define the output HTML file
output_file("data_visualization_dashboard.html")

# Donut Plot for Clusters
cluster_colors = Category10[3]
cluster_wedge_data = prepare_wedge_data('c', cluster_colors)

cluster_plot = figure(height=350, width=350, title="Donut Plot of Clusters", tools='hover',
                      tooltips="@c: @value", x_range=(-1, 1), y_range=(-1, 1))
cluster_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
                   line_color="white", fill_color='color', legend_field='c', source=cluster_wedge_data)
cluster_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
cluster_plot.axis.axis_label = None
cluster_plot.axis.visible = False
cluster_plot.grid.grid_line_color = None

# Donut Plot for Age
age_colors = Category20[len(data['age'].unique())]
age_wedge_data = prepare_wedge_data('age', age_colors)

age_plot = figure(height=350, width=350, title="Donut Plot of Age Groups", tools='hover',
                  tooltips="@age: @value", x_range=(-1, 1), y_range=(-1, 1))
age_plot.wedge(x=0, y=0, radius=0.8, start_angle='start_angle', end_angle='end_angle',
               line_color="white", fill_color='color', legend_field='age', source=age_wedge_data)
age_plot.circle(x=0, y=0, radius=0.4, fill_color='white')
age_plot.axis.axis_label = None
age_plot.axis.visible = False
age_plot.grid.grid_line_color = None

# Create a ColumnDataSource
source = ColumnDataSource(data)

# Define tooltips for hover
TOOLTIPS = [
    ("Frame", "@frame"),
    ("X", "@x"),
    ("Y", "@y"),
    ("Cluster", "@c"),
    ("Age", "@age"),
    ("Loc", "@loc"),
]

# Create the detailed scatter plot
detailed_plot = figure(
    title="Detailed Scatter Plot",
    tools="pan,wheel_zoom,box_zoom,reset,hover",
    width=800,
    height=400,
    tooltips=TOOLTIPS,
    background_fill_color="#efefef"
)
detailed_plot.scatter(x='x', y='y', color='color', source=source, size=10)
detailed_plot.add_tools(HoverTool(tooltips=TOOLTIPS))

# Create the minimap
minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)
minimap.scatter(x='x', y='y', color='color', source=source, size=10)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

# Create and add the RangeTool
range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range)
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
minimap.add_tools(range_tool)

# Create and add the BoxAnnotation for dragging
box = BoxAnnotation(
    left=detailed_plot.x_range.start,
    right=detailed_plot.x_range.end,
    top=detailed_plot.y_range.end,
    bottom=detailed_plot.y_range.start,
    fill_color="lightgrey",
    fill_alpha=0.3,
    editable=True  # Allow the box to be resized
)
minimap.add_layout(box)

# Create a button to toggle the minimap
toggle_button = Button(label="Toggle Minimap", button_type="success")
toggle_button.js_on_click(CustomJS(args=dict(minimap=minimap), code="""
    minimap.visible = !minimap.visible;
"""))

# Create multiple image placeholders
image_placeholder_1 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 1" width="150" height="150">', width=150, height=150)
image_placeholder_2 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 2" width="150" height="150">', width=150, height=150)
image_placeholder_3 = Div(text='<img src="https://via.placeholder.com/150" alt="Image 3" width="150" height="150">', width=150, height=150)

# Add a blurb of text with HTML and LaTeX
text_blurb = Div(text="""
    <h2>Data Visualization Dashboard</h2>
    <p>This dashboard includes various visualizations to help understand the dataset:</p>
    <ul>
        <li>Donut plots representing <b>clusters</b> and <b>age groups</b>.</li>
        <li>A detailed scatter plot with an interactive minimap.</li>
        <li>Multiple image placeholders for relevant graphics.</li>
        <li>Additional graph placeholders for further analysis.</li>
        <li>A video placeholder for visual content.</li>
    </ul>
    <p>Mathematical formula example: \( E = mc^2 \)</p>
""", width=800)

# Additional graph placeholders
additional_plot_1 = figure(height=350, width=350, title="Additional Graph 1")
additional_plot_2 = figure(height=350, width=350, title="Additional Graph 2")

# Arrange the donut plots side by side
donut_plots = row(cluster_plot, age_plot)

# Combine all components into a structured layout
layout = column(
    text_blurb,
    row(image_placeholder_1, image_placeholder_2, image_placeholder_3),
    donut_plots,
    row(additional_plot_1, additional_plot_2),
    detailed_plot,
    minimap,
    toggle_button,
    Div(text='<video width="320" height="240" controls><source src="your_video.mp4" type="video/mp4">Your browser does not support the video tag.</video>', width=320, height=240),
    sizing_mode='stretch_both'  # Allow responsive sizing
)

# Add layout to the document
curdoc().add_root(layout)

# Show the results
show(layout)


<>:123: SyntaxWarning: invalid escape sequence '\('
<>:123: SyntaxWarning: invalid escape sequence '\('
/var/folders/pf/49k8jg7948ddty_lg9b4pybr0000gn/T/ipykernel_68843/623267767.py:123: SyntaxWarning: invalid escape sequence '\('
  text_blurb = Div(text="""


Loading BokehJS ...